# Imports

In [ ]:
import nltk
import pandas as pd

# Activity 3

## 3.1 Sub-activity: Loading and pre-processing of text data

### Task 1

In [ ]:
def get_turing_award_recipients():
    pass

### Task 2

In [ ]:
def get_wikipedia_content():
    pass

### Task 3

In [ ]:
award_winners = {}

### Task 4

### Task 5

## 3.2 Sub-activity: Applying NLP operations on the corpus

### 3.2.1 Stemming

### Task 3

### Task 4

### 3.2.2 Lemmatization

### Task 5

### 3.2.3 Finding synonyms and antonyms

### Task 6

### 3.2.4 Bigrams and trigrams

### Task 7

### Task 8

### Task 9

### Task 10

## 3.3 Sub-section: Visualisation

### 3.3.1 Barplots

### Task 11

### Task 12

### Task 13

### 3.3.2 Heatmap

### Task 14